# Topic modelling analysis of interview transcripts

Load the relevant modules the NLTK stopwords corpus, plus add some custom stopwords

In [13]:
import gensim
import logging
import re
from pprint import pprint
from unidecode import unidecode

# spacy for lemmatization
import spacy

# Import NLTK stop words and add some additional ones
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

import warnings
warnings.filterwarnings('ignore')

Our ASCII data file of collated interview transcripts with interviewer text removed

In [14]:
data_file = "/home/main/Dropbox/data/HtVTranscripts/text/interviewee_megafile.txt"

Read in the file and do some pre-processing of the text.

Namely, converting everything to unicode, making everything lower case, and removing lots of extraneous characters a defined in _spchars_


In [15]:
# Text normalisation
#
def normalize(text):
    spchars = re.compile('\…\`|\~|\!|\@|\#|\$|\%|\^|\&|\*|\(|\)|\_|\+|\=|\\|\||\{|\[|\]|\}|\:|\;|\'|\"|\<|\,|\>|\?|\/|\.|\-')
    text = unidecode(text.decode('utf-8', 'ignore'))    # convert to unicode
    text = text.lower()                                 # convert to lowercase
    text = spchars.sub(" ", text)                       # remove special characters referenced in spchars
    return(text)

# Read input file, normalise
#
def read_input(input_file):
    with open(input_file, 'rb') as f:
        for i, line in enumerate(f):
            if line[0] == 40:                           # Skip any lines than begin with '(' - transcriber's time stamps
                continue
            line = normalize(line)
            yield gensim.utils.simple_preprocess(line)  # Pre-process and return list of words for each line

documents = list(read_input(data_file))

Remove stopwords. Stopwords are common words like 'the', 'is', 'at', 'which' and so on that are high frequency but don't hold a lot of conceptual meaning and so are removed from vector calculations.

We also extend the stopwords list with some common ones from the transcript itself

In [16]:
# Remove stop words
#
def remove_stopwords(texts):
    return [[word for word in gensim.utils.simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

stop_words.extend(['ehm', 'like', 'kinda', 'eh', 'yeah', 'mm', 'roxy'])
documents = remove_stopwords(documents)

Lemmatise corpus. A lemma is the root form of a word. e.g. The lemma for 'was' is 'be'. The lemma for 'apples' is 'apple'. This stops words in the transcript which are conceptually similar or identical ('apple' and 'apples') from being treated as distinct entities.

In [17]:
nlp = spacy.load('en', disable=['ner', 'parser'])
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

documents = lemmatization(documents)

Creation dictionary from corpus and get token frequency

In [19]:
import gensim.corpora as corpora

# Create Dictionary
id2word = corpora.Dictionary(documents)

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in documents]

Build LDA model

In [24]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=50,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

Print topics with association scores

In [25]:
doc_lda = lda_model[corpus]
pprint(lda_model.print_topics())

[(0,
  '0.133*"basically" + 0.114*"sister" + 0.097*"home" + 0.080*"hospital" + '
  '0.036*"music" + 0.030*"live" + 0.025*"attack" + 0.023*"completely" + '
  '0.023*"corner" + 0.022*"rather"'),
 (1,
  '0.121*"feel" + 0.088*"something" + 0.068*"always" + 0.064*"tell" + '
  '0.056*"lot" + 0.056*"want" + 0.037*"stuff" + 0.034*"way" + 0.029*"right" + '
  '0.024*"usually"'),
 (2,
  '0.187*"get" + 0.181*"know" + 0.111*"kind" + 0.060*"sort" + 0.050*"make" + '
  '0.019*"speak" + 0.018*"face" + 0.018*"obviously" + 0.017*"first" + '
  '0.015*"brain"'),
 (3,
  '0.160*"would" + 0.105*"really" + 0.056*"bad" + 0.040*"look" + 0.039*"head" '
  '+ 0.036*"still" + 0.036*"remember" + 0.033*"day" + 0.032*"stop" + '
  '0.031*"fine"'),
 (4,
  '0.167*"go" + 0.112*"say" + 0.092*"thing" + 0.049*"time" + 0.049*"see" + '
  '0.037*"sometimes" + 0.035*"take" + 0.024*"happen" + 0.021*"little" + '
  '0.019*"everything"'),
 (5,
  '0.179*"quite" + 0.133*"work" + 0.109*"mean" + 0.067*"tend" + 0.045*"able" + '
  '0.023*"

Visualise

In [26]:
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.420339 -0.103572       1        1  18.142462
2      0.034868  0.417078       2        1  15.508686
1      0.028189  0.060423       3        1  11.620573
8     -0.038749 -0.039216       4        1  10.681255
7     -0.048199 -0.018040       5        1  10.379860
3     -0.098646 -0.061860       6        1  10.051310
6     -0.065030 -0.092392       7        1   9.537939
9     -0.066946 -0.042368       8        1   7.954062
5     -0.083314 -0.059015       9        1   3.332261
0     -0.082512 -0.061039      10        1   2.791600, topic_info=     Category         Freq        Term        Total  loglift  logprob
87    Default  1504.000000       think  1504.000000  30.0000  30.0000
116   Default  1594.000000         get  1594.000000  29.0000  29.0000
138   Default  1546.000000        know  1546.000000  28.0000  28.0000
36    Default  1665.000000          go  1665.000000  27.0000  27.0000
51    Default   884.000000       would   884.000000  26.0000  26.0000
48    Default  1118.000000         say  1118.000000  25.0000  25.0000
218   Default   945.000000        kind   945.000000  24.0000  24.0000
37    Default   718.000000        hear   718.000000  23.0000  23.0000
191   Default   773.000000        feel   773.000000  22.0000  22.0000
42    Default   919.000000       thing   919.000000  21.0000  21.0000
90    Default   690.000000        well   690.000000  20.0000  20.0000
83    Default   580.000000      really   580.000000  19.0000  19.0000
66    Default   585.000000        come   585.000000  18.0000  18.0000
6     Default   526.000000       voice   526.000000  17.0000  17.0000
86    Default   564.000000   something   564.000000  16.0000  16.0000
1     Default   329.000000       quite   329.000000  15.0000  15.0000
121   Default   445.000000        need   445.000000  14.0000  14.0000
400   Default   449.000000         try   449.000000  13.0000  13.0000
72    Default   512.000000        sort   512.000000  12.0000  12.0000
344   Default   434.000000      always   434.000000  11.0000  11.0000
11    Default   350.000000        talk   350.000000  10.0000  10.0000
212   Default   408.000000        tell   408.000000   9.0000   9.0000
203   Default   340.000000         bit   340.000000   8.0000   8.0000
243   Default   381.000000      people   381.000000   7.0000   7.0000
189   Default   244.000000        work   244.000000   6.0000   6.0000
43    Default   491.000000        time   491.000000   5.0000   5.0000
71    Default   485.000000         see   485.000000   4.0000   4.0000
163   Default   430.000000        make   430.000000   3.0000   3.0000
263   Default   359.000000         lot   359.000000   2.0000   2.0000
88    Default   357.000000        want   357.000000   1.0000   1.0000
...       ...          ...         ...          ...      ...      ...
783   Topic10   148.894821        home   149.549896   3.5742  -2.3348
769   Topic10   123.741837    hospital   124.396805   3.5733  -2.5198
567   Topic10    55.154099       music    55.809334   3.5667  -3.3279
362   Topic10    45.904190        live    46.558788   3.5644  -3.5115
389   Topic10    38.285606      attack    38.940540   3.5616  -3.6929
352   Topic10    35.245289  completely    35.899944   3.5602  -3.7757
596   Topic10    34.999096      corner    35.654747   3.5600  -3.7827
544   Topic10    33.370399      rather    34.025486   3.5591  -3.8304
903   Topic10    31.820866      mental    32.475487   3.5582  -3.8779
546   Topic10    31.201649        city    31.860256   3.5577  -3.8976
875   Topic10    27.708549       black    28.363239   3.5552  -4.0163
1174  Topic10    27.644842     bedroom    28.304749   3.5550  -4.0186
22    Topic10    26.386017      become    27.040691   3.5540  -4.0652
632   Topic10    21.947178     problem    22.601883   3.5492  -4.2494
18    Topic10    19.768461        ring    20.423069   3.5460  -4.3539
876   Topic10    19